In [87]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [88]:
xy = pd.read_csv('배추Model.csv')

def outliers_iqr(data, name) :
  Q1, Q3 = np.percentile(data[name+"가격"], [25, 75])
  IQR = Q3 - Q1
  lower_bound = Q1 - (IQR * 1.5)
  upper_bound = Q3 + (IQR * 1.5)
  outlier_idx = data[name+"가격"][ (data[name+"가격"] < lower_bound) | (data[name+"가격"] > upper_bound) ].index
  data.drop(outlier_idx, axis=0, inplace=True)
  data.reset_index(drop=True, inplace=True)
  return data


xy = outliers_iqr(xy,"배추")



In [89]:
%matplotlib notebook
plt.plot(xy['배추가격']) # 전체 종가
xy.index = xy['날짜']
xy = xy.drop(['날짜'],axis=1)

<IPython.core.display.Javascript object>

In [90]:
seqLength = 21 #window size 
dataDim = 27 # 시가, 고가, 저가, 거래량 , 종가
hiddenDim = 10 
outputDim = 1
lr = 0.01
iterations = 500

trainSize = int(len(xy)*0.7)
trainSet = xy[0:trainSize]
testSet = xy[trainSize-seqLength:]

scaler = MinMaxScaler()

trainSet = scaler.fit_transform(trainSet)
testSet = scaler.transform(testSet)


In [91]:

# 7일간의 5가지 데이터(시가, 종가, 고가, 저가, 거래량)를 받아와서 
# 바로 다음 날의 종가를 예측하는 모델로 구성

def buildDataSet(timeSeries, seqLength):
    xdata = []
    ydata = [] 
    a = len(timeSeries)
    for i in range(0, a-seqLength):
        tx = timeSeries[i:i+seqLength,:-1]
        ty = timeSeries[i+seqLength,[-1]]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)
    
trainX, trainY=buildDataSet(trainSet, seqLength)
testX, testY=buildDataSet(testSet, seqLength)

In [92]:
# 모델 구성 

# First, let's define a RNN Cell, as a layer subclass.
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=[21,26]))

model.add(layers.Dense(1))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 10)                370       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [93]:
# 모델 학습과정 설정 
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 모델 트레이닝 
hist = model.fit(trainX, trainY, epochs=1000, batch_size=16)

Epoch 1/1000
91/91 [==============================] - 1s 3ms/step - loss: 0.0952 - mae: 0.2279
Epoch 2/1000
91/91 [==============================] - 0s 2ms/step - loss: 0.0394 - mae: 0.1545
Epoch 3/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0344 - mae: 0.1439
Epoch 4/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0313 - mae: 0.1376
Epoch 5/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0301 - mae: 0.1350
Epoch 6/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0289 - mae: 0.1319
Epoch 7/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0275 - mae: 0.1290
Epoch 8/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0262 - mae: 0.1254
Epoch 9/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0244 - mae: 0.1214
Epoch 10/1000
91/91 [==============================] - 0s 3ms/step - loss: 0.0210 - mae: 0.1140
Epoch 11/1000
91/91 [============================

KeyboardInterrupt: 

In [ ]:
res = model.evaluate(testX, testY, batch_size=16)
print("loss",res[0],"mae",res[1])

40/40 [==============================] - 0s 1ms/step - loss: 0.6058 - mae: 0.7213
loss 0.605780839920044 mae 0.7212682962417603


In [ ]:
#7 모델 사용
xhat = testX
yhat = model.predict(xhat)

yhat = scaler.inverse_transform(yhat)
testY = scaler.inverse_transform(testY)


In [ ]:
%matplotlib notebook
plt.figure(figsize=(20,10))
plt.plot(yhat, label = "Napa_Cabbage_RNN")
plt.plot(testY,label = "Real")

plt.legend(prop={'size': 20})
plt.savefig('결과/배추모델.png')


<IPython.core.display.Javascript object>